# Урок 3. Классификация. Логистическая регрессия.

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [12]:
def calc_logloss(y, y_pred):
    if 0 in y_pred:
        y_pred[y_pred==0] = 1e-17
    if 1 in y_pred:
        y_pred[y_pred==1] = 1-1e-17
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [16]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    z = np.dot(X, W)
    y_pred = sigmoid(z)
    min_err = calc_logloss(y, y_pred)
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if err < min_err:
            min_err = err
            n_iter = i

    return W, min_err, n_iter, eta

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [17]:
import numpy as np
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [21]:
_, save_min, save_iter, save_eta = eval_model(X_st,y,20000,10)
for ite in [1000,10000,100000]:
    for eta in [1,1e-2,1e-3,1e-4]:
        W, check_min, check_iter, check_eta = eval_model(X_st,y,ite,eta)
        if check_min < save_min:
            save_W, save_min, save_iter, save_eta = W, check_min, check_iter, check_eta
print(save_min, save_iter, save_eta)

/home/mrp/anaconda3/envs/FR/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/home/mrp/anaconda3/envs/FR/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


0.07846357508949717 33949 1


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [19]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W, X.T))
    return y_pred_proba

In [24]:
print(calc_pred_proba(save_W, X_st))

[2.18720281e-01 7.66081142e-06 1.00000000e+00 1.54833279e-16
 9.88519933e-01 7.02747602e-05 1.00000000e+00 2.12771420e-03
 7.94688483e-01 1.00000000e+00]


4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [20]:
def calc_pred(W, X, treshold=0.5):
    y_pred = sigmoid(np.dot(W, X.T)) > treshold
    return y_pred.astype(int)

In [25]:
print(calc_pred(save_W, X_st))

[0 0 1 0 1 0 1 0 1 1]


5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [26]:
def accuracy(y, y_pred):
    count = 0
    for pos, i in enumerate(y):
        if y_pred[pos] == i:
            count += 1
    acc = count / len(y)
    return acc

In [31]:
accuracy([1, 0, 1, 0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

0.9

In [32]:
def err_matrix(y, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for pos, i in enumerate(y):
        if y_pred[pos] == 1 and i == 1:
            TP += 1
        elif y_pred[pos] == 1 and i == 0:
            FP += 1
        elif y_pred[pos] == 0 and i == 1:
            FN += 1
        elif y_pred[pos] == 0 and i == 0:
            TN += 1
        else:
            print('Error')
    df = pd.DataFrame({' ':['𝑎(𝑥) = +1','𝑎(𝑥) = -1'], 'y = +1':[f'TP = {TP}', f'FN = {FN}'], 'y = -1':[f'FP = {FP}', f'TN = {TN}']})
    
    return df.style.hide_index()

In [34]:
import pandas as pd
err_matrix([1, 0, 1, 0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

,y = +1,y = -1
𝑎(𝑥) = +1,TP = 5,FP = 0
𝑎(𝑥) = -1,FN = 1,TN = 4


In [35]:
def precision(y, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for pos, i in enumerate(y):
        if y_pred[pos] == 1 and i == 1:
            TP += 1
        elif y_pred[pos] == 1 and i == 0:
            FP += 1
        elif y_pred[pos] == 0 and i == 1:
            FN += 1
        elif y_pred[pos] == 0 and i == 0:
            TN += 1
        else:
            print('Error')
    return TP / (TP + FP)

In [36]:
precision([1, 0, 1, 0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

1.0

In [37]:
def recall(y, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for pos, i in enumerate(y):
        if y_pred[pos] == 1 and i == 1:
            TP += 1
        elif y_pred[pos] == 1 and i == 0:
            FP += 1
        elif y_pred[pos] == 0 and i == 1:
            FN += 1
        elif y_pred[pos] == 0 and i == 0:
            TN += 1
        else:
            print('Error')
    return TP / (TP + FN)

In [38]:
recall([1, 0, 1, 0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

0.8333333333333334

In [40]:
def f_score(y, y_pred, beta=1):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for pos, i in enumerate(y):
        if y_pred[pos] == 1 and i == 1:
            TP += 1
        elif y_pred[pos] == 1 and i == 0:
            FP += 1
        elif y_pred[pos] == 0 and i == 1:
            FN += 1
        elif y_pred[pos] == 0 and i == 0:
            TN += 1
        else:
            print('Error')
    return (1 + beta ** 2) * TP ** 2 / ((TP + FP) * (TP + FN)) / (beta ** 2 * TP/(TP + FN) + TP / (TP + FN))

In [41]:
f_score([1, 0, 1, 0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

1.0